In [1]:
#!python -m pip install Ipython --upgrade
%load_ext autoreload
%autoreload 2

In [9]:
from models.encoders import VoxNet , CustomResNet50,CustomClassifier
from models.commun_fusion import Concat
from models.mm_models import MMFusion , LitMMFusionModel
import torch 
import torch.nn as nn
from datasets.dataset import data_loader , class_to_index
from pytorch_lightning import Trainer
# useful variable that tells us whether we should use the GPU
use_cuda = torch.cuda.is_available()

if not use_cuda:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...') 
DATASET_FOLDER = "../small_dataset/"
# Define data folders
image_folder = DATASET_FOLDER+'image'
voxel_folder = DATASET_FOLDER+'velodyne'
label_folder = DATASET_FOLDER+'label'
fast_dev_run= True  
epochs= 10
classes = list(class_to_index.keys())
lr= 0.001

CUDA is available!  Training on GPU ...


In [10]:
train_data_loader=data_loader(image_folder,voxel_folder,label_folder,batch_size=16) 

In [11]:
image_backbone = CustomResNet50(num_classes=len(class_to_index),
                                input_channels=3)
point_cloud_backbone = VoxNet()
classifier=CustomClassifier(hidden_dim=0,activation_fun=nn.ReLU,
                            num_class=len(class_to_index),dropout_rate=None)
encoders= [image_backbone,point_cloud_backbone]
fusion = Concat()
model_fusion = MMFusion(encoders,fusion,classifier)

hidden_dim is 0


In [14]:
import pytorch_lightning as pl
pl.__version__

'1.9.5'

In [12]:
#model_path = os.path.join(os.getcwd(), f'{cfg.output_folder}{name_experiment}')

#checkpoint_callback = callbacks.ModelCheckpoint(dirpath=model_path, save_top_k=1 ,monitor="val_acc",mode="max")

#early_stopping = callbacks.EarlyStopping(monitor="val_acc",mode="max",patience=10,strict=False)
optimizer = torch.optim.Adam(model_fusion.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
litmodel=LitMMFusionModel(model_fusion,classes,criterion,lr)

trainer = Trainer(max_epochs=epochs,fast_dev_run=fast_dev_run,
                  auto_lr_find=True,deterministic=True,accelerator='gpu', 
                  devices=1)

trainer.fit(litmodel,train_dataloaders=train_data_loader,val_dataloaders=None)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
You are using a CUDA device ('NVIDIA GeForce RTX 3080 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | MMFusion         | 53.3 M
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
53.3 M    Trainable params
0         Non-trainable params
53.3 M    Total params
213.309

Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] voxel data shape torch.Size([32, 32, 32])
voxel data shape torch.Size([32, 32, 32])
voxel data shape torch.Size([32, 32, 32])
voxel data shape torch.Size([32, 32, 32])
voxel data shape torch.Size([32, 32, 32])
voxel data shape torch.Size([32, 32, 32])
padded voxel data in fn shape is  torch.Size([6, 32, 32, 32])
x in input resnet torch.Size([6, 3, 224, 224])
x shape before flattening  torch.Size([6, 1000])
in custom resnet x shape is  torch.Size([6, 1000])
in forward  torch.Size([6, 32, 32, 32])
torch.Size([6, 1000]) torch.Size([64, 3375])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 6 but got size 64 for tensor number 1 in the list.

In [13]:
oup = VoxNet()
data = torch.rand([6, 32, 32, 32])
# Pass the input through the backbone
output_features = oup(data)
print("Output shape:", output_features.shape)

in forward  torch.Size([6, 32, 32, 32])
Output shape: torch.Size([64, 3375])


In [ ]:
# Training loop
num_epochs = 10
log_interval = 10

for epoch in range(num_epochs):
    concat_model.train()
    for batch_idx, (images, voxel_data, labels) in enumerate(train_data_loader):
        print("Images shape:", images.shape)
        print("Voxel data shape:", voxel_data.shape)
        batch_size = images.size(0)
        labels = torch.randint(8, (batch_size,))
        optimizer.zero_grad()
        # Pass images through the image backbone
        image_features = image_backbone(images)
        print("Image features shape:", image_features.shape)

        # Pass voxel data through the voxel backbone
        voxel_features = point_cloud_backbone(voxel_data)

        # Flatten the voxel features
        voxel_features = voxel_features.view(voxel_features.size(0), -1)

        # Print shapes of intermediate features
        print("Voxel features shape:", voxel_features.shape)

        batch_size = image_features.size(0)
        voxel_features_reshaped = voxel_features[:batch_size, :]
        # Concatenate features
        concat_model = ConcatenationNetwork(image_backbone, point_cloud_backbone)
        output = concat_model(images, voxel_data)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_data_loader)}], Loss: {loss.item()}')

print('Training complete')

Images shape: torch.Size([6, 3, 224, 224])
Voxel data shape: torch.Size([6, 32, 32, 32])
Image features shape: torch.Size([6, 8])
Voxel features shape: torch.Size([64, 3375])
Epoch [1/10], Batch [1/1], Loss: 2.0889408588409424
Images shape: torch.Size([6, 3, 224, 224])
Voxel data shape: torch.Size([6, 32, 32, 32])
Image features shape: torch.Size([6, 8])
Voxel features shape: torch.Size([64, 3375])
Epoch [2/10], Batch [1/1], Loss: 2.100167751312256
Images shape: torch.Size([6, 3, 224, 224])
Voxel data shape: torch.Size([6, 32, 32, 32])
Image features shape: torch.Size([6, 8])
Voxel features shape: torch.Size([64, 3375])
Epoch [3/10], Batch [1/1], Loss: 2.0702569484710693
Images shape: torch.Size([6, 3, 224, 224])
Voxel data shape: torch.Size([6, 32, 32, 32])
Image features shape: torch.Size([6, 8])
Voxel features shape: torch.Size([64, 3375])
Epoch [4/10], Batch [1/1], Loss: 2.09112286567688
Images shape: torch.Size([6, 3, 224, 224])
Voxel data shape: torch.Size([6, 32, 32, 32])
Image 

In [ ]:
def custom_weight_init(m):
    classname = m.__class__.__name__
    # Intiatize weights for Linear and Conv2D layers
    if classname.find('Linear') != -1:
        n_inputs =  m.in_features
        y = 1/np.sqrt(n_inputs)
        m.weight.data.normal_(0, y)
        m.bias.data.fill_(0)

concat_model.apply(custom_weight_init)